In [1]:
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util

%matplotlib inline
import matplotlib.pyplot as plt

from coffea.nanoaod import NanoEvents

In [2]:
from pyinstrument import Profiler
profiler = Profiler()
profiler.start()

from coffea.nanoaod import NanoEvents
from coffea.nanoaod.methods import FatJet, LorentzVector

arraycache = {}
events = NanoEvents.from_file(
    # 'data/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8.root',
    # 'root://cmsxrootd-site.fnal.gov//store/mc/RunIIFall17NanoAODv6/QCD_Pt_50to80_TuneCP5_13TeV_pythia8/NANOAODSIM/PU2017_12Apr2018_Nano25Oct2019_102X_mc2017_realistic_v7-v1/280000/FDFE267A-9BCF-6347-A4DE-CC108EEAC18E.root',
    # 'data/EGamma-Run2018D.root',
    # 'data/TTToHadronic_TuneCP5_13TeV-powheg-pythia8_10X.root',
    #'data/RunIIAutumn18NanoAODv5_BulkGravTohhTohWWhbb.root',
    'data/NanoAOD_Oct16_1.root',
    #entrystop=20000,
    cache=arraycache,
    methods={
        'FatJetPFCands': LorentzVector,
    },
)
print(arraycache)

profiler.stop()

{}


In [ ]:
#%%timeit

profiler = Profiler()
profiler.start()

gen = events.GenPart

where = (
    (abs(gen.pdgId) == 5)
    & (np.abs(gen.distinctParent.pdgId) == 6).fillna(False)
    & gen.hasFlags(['isHardProcess'])
)
genb = gen[where]

gentop = genb.distinctParent
hasWmu = (
    (abs(gentop.children.pdgId) == 24)
    & (abs(gentop.children.children.pdgId) == 13).any()
).any()

chosenb = genb[hasWmu]

profiler.stop()

In [ ]:
with open('profile.html', 'w') as fout:
    fout.write(profiler.output_html())

In [ ]:
(abs(gentop.children.pdgId) == 24).count()

In [ ]:
gentop.pdgId

In [ ]:
gentop.children.pdgId

In [ ]:
x = np.unique(chosenb.counts, return_counts=True)
x[0], x[1] / 20000

In [ ]:
genele = events.GenPart[
    (abs(events.GenPart.pdgId) == 11)
    & events.GenPart.hasFlags(['isHardProcess', 'isLastCopy'])
]
genele_fromW = genele[
    (genele.distinctParent.pdgId == 24)
]
genW = genele_fromW.distinctParent

In [ ]:
leadingmuon = events.Muon.pad(1, clip=True).flatten()
leadingjet = events.CustomAK8Puppi.pad(1, clip=True).flatten()
met = events.MET

In [ ]:
from uproot_methods import TLorentzVectorArray

In [ ]:
joffshell = leadingjet.msoftdrop < 125/2  # halfway point between offshell and onshell W
massassumption = 80.*joffshell + (125 - 80.)*~joffshell
x = massassumption**2/(2*leadingmuon.pt*met.rho) + np.cos(leadingmuon.phi - met.phi)
met_eta = (
    (x < 1)*np.arcsinh(x*np.sinh(leadingmuon.eta))
    + (x > 1)*(
        leadingmuon.eta
        - np.sign(leadingmuon.eta)*np.arccosh(leadingmuon.eta)
    )
)
met_p4 = TLorentzVectorArray.from_ptetaphim(met.rho, met_eta.fillna(0.), met.phi, np.zeros(met.size))

In [ ]:
leadingmuon

In [ ]:
plt.hist(met_p4.eta);

In [ ]:
bosons = events.GenPart[
    (np.abs(events.GenPart.pdgId) >= 21)
    & (np.abs(events.GenPart.pdgId) <= 37)
    & events.GenPart.hasFlags(['isHardProcess', 'isLastCopy'])
]
candidatejet = events.CustomAK8Puppi[:,:1]
ak8_boson_pair = candidatejet.cross(bosons, nested=True)
ak8_boson_pair['dR2'] = ak8_boson_pair.i0.delta_r2(ak8_boson_pair.i1)
ak8_boson_pair['dPt2'] = ((ak8_boson_pair.i0.pt - ak8_boson_pair.i1.pt)/(ak8_boson_pair.i0.pt + ak8_boson_pair.i1.pt))**2

In [ ]:
matchedBoson = ak8_boson_pair.i1[(ak8_boson_pair.dR2 + ak8_boson_pair.dPt2).argmin()].flatten(axis=1)
plt.hist(matchedBoson.pdgId.flatten(), bins=np.arange(20, 30)-.5);

In [ ]:
fatjets = events.FatJet
from boostedhiggs import corrections

fatjets['msdcorr'] = corrections.corrected_msoftdrop(fatjets)
fatjets['rho'] = 2*np.log(fatjets.msdcorr / fatjets.pt)
fatjets['n2ddt'] = fatjets.n2b1 - corrections.n2ddt_shift(fatjets, year='2018')

In [ ]:
plt.hist(fatjets.n2ddt.flatten(), bins=np.linspace(-0.5, 0.5, 50));

In [ ]:
leadingjet = fatjets[:, 0:1]

good = ((leadingjet.pt > 450)
        & (np.abs(leadingjet.eta) < 2.4)
        & (leadingjet.jetId & 2).astype(bool)  # tight
        & (leadingjet.n2ddt < 0.)
        & (leadingjet.msdcorr > 40)
       )

good_event = good.all()

In [ ]:
jets = events.Jet
ak4_ak8_pair = jets.cross(leadingjet, nested=True)
dphi = ak4_ak8_pair.i0.delta_phi(ak4_ak8_pair.i1)
opp = (np.abs(dphi) > np.pi / 2).all()
# only consider first 4 jets to be consistent with old framework
ak4_opposite = jets[opp & (jets.pt > 30.) & (jets.array.localindex < 4)]

btagLooseWPs = {
    '2016': 0.6321,
    '2017': 0.4941,
    '2018': 0.4184,
}
good_event &= ak4_opposite.btagDeepB.max() < btagLooseWPs['2018']

In [ ]:
nelectrons = ((events.Electron.cutBased >= events.Electron.LOOSE) & (events.Electron.pt > 20)).counts
nmuons = (events.Muon.looseId & (events.Muon.pt > 20)).counts
ntaus = (events.Tau.idDecayMode & (events.Tau.pt > 20)).counts
good_event &= (events.MET.pt < 140.) & (nmuons == 0) & (nelectrons == 0) & (ntaus == 0)

In [ ]:
print("Passed %d of %d events" % (good_event.sum(), good_event.size))

In [ ]:
list(arraycache.keys())

In [ ]:
events.Muon.matched_gen.pdgId

In [ ]:
lscut = (events.CustomAK8Puppi.muonIdx3SJ >= 0)
events = events[lscut.any()]
lscut = lscut[lscut.any()]

In [ ]:
cut = lscut #& (events.CustomAK8Puppi.pt - events.CustomAK8Puppi.LSpt < 5.)

In [ ]:
lscut = (events.CustomAK8Puppi.muonIdx3SJ >= 0)
events = events[lscut.any()]
lscut = lscut[lscut.any()]
cut = lscut 
print(len(events.CustomAK8Puppi.muonIdx3SJ))
print(len(events.Muon[events.CustomAK8Puppi[cut].muonIdx3SJ.flatten()]))
print(len(events.CustomAK8Puppi[cut]))
ak8_muon_pair = events.Muon[events.CustomAK8Puppi[cut].muonIdx3SJ.flatten()].cross(events.CustomAK8Puppi[cut])
mujet_dR = ak8_muon_pair.i0.delta_r(ak8_muon_pair.i1)


In [ ]:
plt.hist(
    events.Muon[events.CustomAK8Puppi[cut].muonIdx3SJ.flatten()].delta_r(events.CustomAK8Puppi[cut]).flatten(),
    bins=np.linspace(0,2,50),
);

In [ ]:
jet = events.CustomAK8Puppi[lscut]
mu = events.Muon[jet.muonIdx3SJ]
cut = (mu.pt > 20) & (jet.lsf3 > 0.1) & (abs(mu.matched_gen.distinctParent.pdgId) == 24)

In [ ]:
#plt.hist(jet[cut].LSmsoftdrop.flatten(), bins=np.linspace(0, 200, 50), alpha=0.5, label='LS msoftdrop');
plt.hist(jet[cut].msoftdrop.flatten(), bins=np.linspace(0, 200, 50), alpha=0.5, label='Jet msoftdrop');
plt.hist((jet - mu)[cut].mass.flatten(), bins=np.linspace(0, 200, 50), alpha=0.5, label='Jet - muon mass');
plt.gca().legend()

In [ ]:
events.materialized

In [ ]:
events.size

In [ ]:
goodmuon = (
    (events.Muon.pt > 10)
    & (np.abs(events.Muon.eta) < 2.4)
    & (events.Muon.sip3d < 4)
    & (np.abs(events.Muon.dz) < 0.1)
    & (np.abs(events.Muon.dxy) < 0.05)
    & (events.Muon.mvaId == 2)
            )
nmuons = goodmuon.sum()
leadingmuon = events.Muon[goodmuon][:, 0:1]
ak8_muon_pair = leadingmuon.cross(events.CustomAK8Puppi.array)
ak8_muon_dR = ak8_muon_pair.i0.delta_r(ak8_muon_pair.i1)
mu_in_cone = ak8_muon_dR.min() < 0.8 # this I am not sure we have to put as a selection...                                                                                      
bestidx = ak8_muon_dR.argmin()
leadingjet = events.CustomAK8Puppi[bestidx]

jets = events.Jet.array[(events.Jet.pt > 30.) & (events.Jet.jetId & 2)]
ak4_ak8_pair = jets.cross(leadingjet, nested=True)
ak4_ak8_dphi = ak4_ak8_pair.i0.delta_phi(ak4_ak8_pair.i1)
ak4_opposite = jets[(np.abs(ak4_ak8_dphi) > np.pi / 2).all()]

nelectrons = (
            (events.Electron.pt > 10)
            & (np.abs(events.Electron.eta) < 2.5)
            & (events.Electron.cutBased & (1 << 2)).astype(bool)  # 2017V2 loose                                                                                                    
        ).sum()

from uproot_methods import TLorentzVectorArray

leadingjet_p4 = TLorentzVectorArray.from_ptetaphim(leadingjet.pt, leadingjet.eta, leadingjet.phi, leadingjet.msoftdrop)
leadingmuon_p4 = TLorentzVectorArray.from_ptetaphim(leadingmuon.pt,leadingmuon.eta,leadingmuon.phi, leadingmuon.mass)
print(leadingjet_p4.pad(1, clip=False))
print(leadingmuon_p4.pad(1, clip=False))
mm = (leadingjet_p4 - leadingmuon_p4).mass2


In [ ]:
print(leadingmuon)

In [ ]:
#leadingmuon = events.Muon[:,0:1]
ak8_muon_pair = leadingmuon.cross(events.CustomAK8Puppi.array)
mujet_dR = ak8_muon_pair.i0.delta_r(ak8_muon_pair.i1)
mu_in_cone = mujet_dR.min() < 0.8
bestidx = mujet_dR.argmin().flatten()
leadingjet = events.CustomAK8Puppi[bestidx].flatten()
met = events.MET

In [ ]:
lscut = (events.CustomAK8Puppi.muonIdx3SJ >= 0)
events = events[lscut.any()]
lscut = lscut[lscut.any()]
jet = events.CustomAK8Puppi[lscut]
mu = events.Muon[jet.muonIdx3SJ]
print(jet.muonIdx3SJ.flatten())
print(events.Muon.array)
mu = events.Muon[jet.muonIdx3SJ]
cut = (mu.pt > 20) & (jet.lsf3 > 0.1) & (abs(mu.matched_gen.distinctParent.pdgId) == 24)